In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!unzip "drive/MyDrive/Colab Notebooks/datasets/kagglecatsanddogs_3367a.zip" -d "drive/MyDrive/Colab Notebooks/datasets/" -y

In [4]:
import cv2, os
import numpy as np
from tqdm import tqdm

REBUILD_DATA = False

class DogsVCats():
    img_size = 50
    cats = "drive/MyDrive/Colab Notebooks/datasets/PetImages/Cat"
    dogs = "drive/MyDrive/Colab Notebooks/datasets/PetImages/Dog"
    labels = {cats: 0, dogs: 1}

    training_data = []
    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.labels:
            print(label)
            for image in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, image)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.img_size, self.img_size))
                    self.training_data.append([np.array(img), 
                                              np.eye(2)[self.labels[label]]])
                    
                    if label == self.cats:
                      self.catcount += 1
                    elif label == self.dogs:
                      self.dogcount += 1
                
                except Exception as e:
                    pass
        
        np.random.shuffle(self.training_data)
        np.save("drive/MyDrive/Colab Notebooks/datasets/training_data.npy", self.training_data)
        print("Cats: ", self.catcount)
        print("Dogs: ", self.dogcount)

if REBUILD_DATA:
    dogsvcats = DogsVCats()
    dogsvcats.make_training_data()


In [5]:
training_data = np.load("drive/MyDrive/Colab Notebooks/datasets/training_data.npy", allow_pickle=True)

In [6]:
print(len(training_data))
print(training_data.shape)

24946
(24946, 2)


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5) #Input, Output, Kernel size
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)

        self.fc1 = nn.Linear(128*8*8,512)
        self.fc2 = nn.Linear(512,2)

        #x = torch.randn(50,50).view(-1,1,50,50)
        #self._to_linear = None

    def forward(self, x):
        x = self.conv1(x) #Output is (32 x 46 x 46)
        x = F.relu(x)
        x = self.conv2(x) #Output is (64 x 42 x 42)
        x = F.relu(x)
        x = F.max_pool2d(x,2) #Output is (64 x 21 x 21)
        x = self.conv3(x)     #Output is (128 x 17 x 17)
        x = F.relu(x)
        x = F.max_pool2d(x,2) #Output is (128 x 8 x 8)
        x = torch.flatten(x,1) 
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

optimizer = optim.Adam(net.parameters(), lr= 0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

Val_Pct = 0.1
val_size = int(len(X)*Val_Pct)
#print(val_size)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

#print(len(train_X))
#print(len(test_X))

net = Net().to(device)

def train(net):
  batch_size = 100
  EPOCHS = 10

  for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X),batch_size)):
        #print(i, i+batch_size) 
        batch_X = train_X[i:i+batch_size].view(-1,1,50,50).to(device)
        batch_y = train_y[i:i+batch_size].to(device)

        net.zero_grad()
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step() # Does the update

    print(f"Epoch: {epoch}, loss: {loss}")

def test(net):
  correct = 0
  total = 0
  with torch.no_grad():
      for i in tqdm(range(len(test_X))):
          real_class = torch.argmax(test_y[i]).to(device)
          net_out = net(test_X[i].view(-1,1,50,50).to(device))[0]

          predicted_class = torch.argmax(net_out)
          if predicted_class == real_class:
              correct += 1
          total += 1

  print("Accuracy:", round(correct/total,3))


train(net)
test(net)

Running on GPU


  2%|▏         | 5/225 [00:00<00:04, 45.01it/s]

Epoch: 0, loss: 1.6750696897506714


  2%|▏         | 5/225 [00:00<00:05, 43.96it/s]

Epoch: 1, loss: 1.6750696897506714


...

Epoch: 8, loss: 1.6750696897506714


 12%|█▏        | 305/2494 [00:00<00:01, 1509.72it/s]

Epoch: 9, loss: 1.6750696897506714


100%|██████████| 2494/2494 [00:01<00:00, 1498.42it/s]

Accuracy: 0.495


In [13]:
torch.cuda.is_available()
torch.cuda.device_count()

1

In [11]:
device = torch.device("cuda:0")

device(type='cuda', index=0)

In [25]:
net.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=8192, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)